Prompt Chaining

In [ ]:
# ✅ Imports
from langgraph.graph import StateGraph, START, END   # LangGraph workflow builder
from langchain_huggingface import HuggingFacePipeline  # Hugging Face → LangChain wrapper
from typing import TypedDict   # Define structured state schema
from dotenv import load_dotenv # Load env variables (e.g., Hugging Face token)

import torch, gc

# ---------------------------------------------------
# 1. Free GPU memory before loading the model
# ---------------------------------------------------
gc.collect()                  # Run Python garbage collector
torch.cuda.empty_cache()      # Release cached but unused GPU memory
torch.cuda.ipc_collect()      # Clean up interprocess GPU memory handles

# ---------------------------------------------------
# 2. Load environment variables
# ---------------------------------------------------
# This pulls values from your .env file (like HF API token).
load_dotenv()

# ---------------------------------------------------
# 3. Choose and wrap an LLM model
# ---------------------------------------------------
gemma_model = "google/gemma-2-2b-it"  # Instruction-tuned Gemma model

# HuggingFacePipeline wraps a HF text-generation pipeline
# so it behaves like a LangChain LLM object.
model = HuggingFacePipeline.from_model_id(
    model_id=gemma_model,
    task="text-generation",   # Pipeline type
    pipeline_kwargs=dict(
        temperature=0.5,      # Lower = more deterministic
        max_new_tokens=100,   # Limit output length
        return_full_text=False # Return only generated text (not prompt+text)
    ),
)

# ---------------------------------------------------
# 4. Define the workflow state schema
# ---------------------------------------------------
# Each node in the LangGraph will pass this dictionary around.
class BlogState(TypedDict):
    title: str    # Input: blog post title
    outline: str  # Intermediate: outline generated from title
    content: str  # Final: full blog content


# ---------------------------------------------------
# 5. Define workflow nodes
# ---------------------------------------------------

# Node 1: create_outline
def create_outline(state: BlogState) -> BlogState:
    # Get the title from state
    title = state['title']
    
    # Prompt the model to generate a blog outline
    prompt = f"Create a detailed outline for a blog post titled: {title}"
    
    outline = model.invoke(prompt)   # Call the model
    
    state['outline'] = outline       # Save output in state
    return state


# Node 2: create_blog
def create_blog(state: BlogState) -> BlogState:
    # Get inputs from state
    title = state['title']
    outline = state['outline']
    
    # Prompt the model to generate full blog content based on the outline
    prompt = f"Write a detailed blog post based on the title: {title} and the outline: {outline}"
    
    content = model.invoke(prompt)
    
    state['content'] = content       # Save output in state
    return state


# ---------------------------------------------------
# 6. Build the workflow graph
# ---------------------------------------------------
graph = StateGraph(BlogState)       # Initialize with our state schema

# Register nodes
graph.add_node('create_outline', create_outline)
graph.add_node('create_blog', create_blog)

# Define execution flow:
# START → create_outline → create_blog → END
graph.add_edge(START, 'create_outline')
graph.add_edge('create_outline', 'create_blog')
graph.add_edge('create_blog', END)

# Compile graph into an executable workflow
workflow = graph.compile()

# ---------------------------------------------------
# 7. Run the workflow
# ---------------------------------------------------
initial_state = {'title': "The Future of AI in Healthcare"}  # Input to workflow

# Run graph execution
final_state = workflow.invoke(initial_state)

# ---------------------------------------------------
# 8. Show final blog content
# ---------------------------------------------------
print(final_state['content'])

/home/lisa/anaconda3/envs/llm_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [01:05<00:00, 32.54s/it]
Device set to use cuda:0
/home/lisa/anaconda3/envs/llm_env/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/lisa/anaconda3/envs/llm_env/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `tem

 and Treatment:**
        * Explain how AI can analyze medical images (X-rays, CT scans, etc.) to detect diseases earlier and more accurately.
        * Discuss AI-powered tools for personalized medicine, tailoring treatments to individual patients.
        * Highlight the potential of AI in drug discovery and development.
    * **B. Enhanced Efficiency and Accessibility:**
        * Explain how AI can automate administrative tasks, freeing up healthcare professionals for patient care.
        * Discuss the potential of


In [2]:
print(final_state['outline'])


: Promises and Perils.

**I. Introduction**
    * Hook: Start with a compelling statistic or anecdote about AI's impact on healthcare.
    * Briefly introduce the topic: AI's growing role in healthcare and its potential benefits and risks.
    * Thesis statement: AI holds immense promise for revolutionizing healthcare, but its implementation requires careful consideration of ethical and practical challenges.

**II. The Promise of AI in Healthcare**
    * **A. Improved Diagnosis
